## Program for downloading the CATH database
---

Previous libraries that need to be downloaded.

To start working, it will be necessary to create a folder to download the files and change our working directory, going to the folder we have created.

***If you want to download the complete database, type variable="YES", otherwise, if you want the test version, type variable = "NO". 
The complete database requires about 40GB.***

In [68]:
import os 
import pandas as pd
import wget
import requests
import json

datadir=os.path.expanduser('~/data')
RCSB_pdb=os.path.expanduser('~/data/RCSB_pdb')
variable="NO"

In [69]:
if os.path.exists(datadir)== False:
    os.mkdir(datadir)
    os.chdir(datadir)
    os.mkdir(RCSB_pdb)
else:
    os.chdir(datadir)

In [70]:
os.getcwd() 

'/home/joanc/data'

To access the information found in CATH, you need the following link and then we can proceed to download.
* Url: ftp://orengoftp.biochem.ucl.ac.uk/cath/releases/latest-release/cath-classification-data/ "file"

In [65]:
!wget -nc ftp://orengoftp.biochem.ucl.ac.uk/cath/releases/latest-release/cath-classification-data/cath-domain-description-file.txt


!wget -nc ftp://orengoftp.biochem.ucl.ac.uk/cath/releases/latest-release/cath-classification-data/cath-names.txt

El fichero “cath-domain-description-file.txt” ya está ahí, no se recupera.
El fichero “cath-names.txt” ya está ahí, no se recupera.


---
### We proceed to work with the files.

We will start by creating a dictionary, which links the code that cath uses with the different hierarchies.

In [66]:
def read_file(file):            
    f=open(file,"r")            
    list=f.readlines()          
    f.close()                   
    return(list)
def create_dic(list):
    d={}
    d2={}
    for e in list:
        if e[0]!='#':
            v1=e.find("    ")
            cad=e[v1+4:]
            v2=cad.find('    ')
            codeCATH=e[:v1]
            codePDB=cad[:v2-3]  #we extract the last three characters because they are not part of the pdb code.
            node=cad[v2+5:-1]
            d[codeCATH]=codePDB
            d2[codeCATH]=node
    return(d,d2)

In [67]:
list_Hierarchy= read_file("./cath-names.txt") # We open the file containing the names of the different hierarchies that exist.
(dic_pdb,dic_node)=create_dic(list_Hierarchy)
dic_pdb
#dic_node

{'1': '1xmk',
 '2': '4unu',
 '3': '1hdo',
 '4': '1g6x',
 '6': '1vwx',
 '1.10': '1xmk',
 '1.20': '2ynz',
 '1.25': '3ro3',
 '1.40': '3iis',
 '1.50': '4ayo',
 '2.10': '3sov',
 '2.20': '6d9n',
 '2.30': '2o9s',
 '2.40': '1x54',
 '2.50': '4mxt',
 '2.60': '4unu',
 '2.70': '1rwh',
 '2.80': '3nbc',
 '2.90': '2dpf',
 '2.100': '3ll2',
 '2.102': '2bmo',
 '2.105': '1n7v',
 '2.110': '3c7x',
 '2.115': '4n1i',
 '2.120': '1pjx',
 '2.130': '2oiz',
 '2.140': '4a5s',
 '2.150': '5d7w',
 '2.160': '1k5c',
 '2.170': '3f9x',
 '2.180': '5kis',
 '3.10': '3goe',
 '3.15': '1ewf',
 '3.20': '2vxn',
 '3.30': '6n3d',
 '3.40': '1hdo',
 '3.50': '1lqt',
 '3.55': '4jtm',
 '3.60': '5xp6',
 '3.65': '5u4h',
 '3.70': '1ud9',
 '3.75': '6nib',
 '3.80': '4fs7',
 '3.90': '2w8t',
 '3.100': '1vq8',
 '4.10': '1g6x',
 '6.10': '1vwx',
 '6.20': '3hrz',
 '1.10.8': '1oai',
 '1.10.10': '1xmk',
 '1.10.12': '5jbx',
 '1.10.15': '4c1n',
 '1.10.20': '4cay',
 '1.10.30': '3fgh',
 '1.10.40': '2ptr',
 '1.10.45': '1wvf',
 '1.10.60': '1on2',
 '1.10.

The following is an example of how to access the UniProt code using the PDB code

It is necessary to create a third dictionary that links the CATH code with the UniProt code. To obtain the UniProt code we will access the RCSB PDB web page of each protein. It will be necessary to use the following url:
* https://www.rcsb.org/structure/protein_code_pdb


In [57]:
def update_pdb_code(code_pdb,result):

    while result== None: 
        url="https://www.rcsb.org/structure/" + code_pdb
        #print(url)
        page= requests.get(url)
        soup= BeautifulSoup(page.content,'html.parser')
        result= soup.find("li",{"id":"note_obsoletedBy"})
        if result!=None:
            #print(result)
            dirty_code=result.text
            #print(dirty_code)
            code_pdb=dirty_code[-5:-1]
            url="https://www.rcsb.org/structure/" + code_pdb
            #print(url)
            page= requests.get(url)
            soup= BeautifulSoup(page.content,'html.parser')
            result= soup.find("div",{"class":"col-lg-2 col-md-2 text-right"})
            #print(result)
        else:
            #print("there is no UniProt code for this protein")
            code_UniProt="there is no UniProt code for this protein"
            #print(result)
            break

    if result!=None:
        #print(dirty_code)
        dirty_code=result.text
        v=dirty_code.find(":")
        code_UniProt=dirty_code[v+3:]
    return(code_pdb,code_UniProt)

def find_UniProt_code(dic_pdb):
    from bs4 import BeautifulSoup
    dic_UniProt={}

    for k in dic_pdb:
        code_pdb=dic_pdb[k]
        url="https://www.rcsb.org/structure/" + code_pdb
        #print(url)
        page= requests.get(url)
        soup= BeautifulSoup(page.content,'html.parser')
        try:
            result= soup.find("div",{"class":"col-lg-2 col-md-2 text-right"})
            dirty_code=result.text
            v=dirty_code.find(":")
            code=dirty_code[v+3:]
            dic_UniProt[code_pdb]=code
            #print(code)
        except AttributeError:
            (code_PDB,code)=update_pdb_code(code_pdb,result)    #this function allows to find the updates of the pdb codes and thus to obtain the corresponding UniProt code.
            dic_UniProt[code_PDB]=code
            dic_pdb[k]=code_PDB # +"nnn"    #3 characters are added because all the pdb codes in the dictionary contain them (this problem will be solved later).
    return(dic_UniProt)

In [54]:
def write_file(fileName,dic):
    file = open(fileName, "w")
    json.dump(dic, file)
    file.close()
def read_file2(fileName):
    with open(fileName, 'r') as f:
        json_data = json.load(f)
    return(json_data)

if os.path.exists("dic_UniProt.json")==False:       #if the file with the UniProt dictionary information already exists, it does not execute the code.
    dic_UniProt=find_UniProt_code(dic_pdb)
    write_file("dic_UniProt.json",dic_UniProt)
else:
    dic_UniProt=read_file2("dic_UniProt.json")

---

### We proceed to extract the data from the pdb and AlphaFold web

We will need the following links, where they will be completed with the program:

* url: https://files.rcsb.org/view/CodePDB.pdb
* url: https://alphafold.ebi.ac.uk/files/AF-CodeUniProt-F1-predicted_aligned_error_v2.json
* url: https://alphafold.ebi.ac.uk/files/AF-CodeUniProt-F1-model_v2.pdb 

We will use the dictionary created earlier.

In [55]:
os.chdir(RCSB_pdb)

In [58]:
#### In process
def download_file(dic,dic2,n):
    for k in dic:
        if k.count(".")==n:
            pdb=dic[k]#[:-3]
            print(dic[k])
            if pdb in dic2:

                CodeUniProt=dic2[pdb]
                print(CodeUniProt)

                code_PDB=pdb+".pdb"
                url_pdb='https://files.rcsb.org/view/' + code_PDB
                
                code_file= 'AF-' + CodeUniProt + '-F1-predicted_aligned_error_v2.json'
                url_file='https://alphafold.ebi.ac.uk/files/' + code_file
                print(url_file)

                code_AlphaFold="AF-" + CodeUniProt + "-F1-model_v2.pdb"
                url_AlphaFold_pdb="https://alphafold.ebi.ac.uk/files/" + code_AlphaFold
                print(url_AlphaFold_pdb)

                try:
                    if os.path.exists(code_PDB)==False:
                        file_name=wget.download(url_pdb)
                    
                    if os.path.exists(code_file)==False:
                        file_name=wget.download(url_file)
                    
                    if os.path.exists(code_AlphaFold)==False:
                        file_name=wget.download(url_AlphaFold_pdb)
                except:
                    print("codi no executat a Alphafold")

v=variable.lower()
if v=="yes":
    n=3 # download the complete database, in this case the information of 6631 proteins is downloaded.
else:
    n=1 # download the test database, in this case the information of 43 proteins is downloaded.

download_file(dic_pdb,dic_UniProt,n) 

1xmkA00
P55265
https://alphafold.ebi.ac.uk/files/AF-P55265-F1-predicted_aligned_error_v2.json
https://alphafold.ebi.ac.uk/files/AF-P55265-F1-model_v2.pdb
2ynzC01
P03069
https://alphafold.ebi.ac.uk/files/AF-P03069-F1-predicted_aligned_error_v2.json
https://alphafold.ebi.ac.uk/files/AF-P03069-F1-model_v2.pdb
3ro3A00
Q8VDU0
https://alphafold.ebi.ac.uk/files/AF-Q8VDU0-F1-predicted_aligned_error_v2.json
https://alphafold.ebi.ac.uk/files/AF-Q8VDU0-F1-model_v2.pdb
3iisM00
P80484
https://alphafold.ebi.ac.uk/files/AF-P80484-F1-predicted_aligned_error_v2.json
https://alphafold.ebi.ac.uk/files/AF-P80484-F1-model_v2.pdb
4ayoA00
B0SWV2
https://alphafold.ebi.ac.uk/files/AF-B0SWV2-F1-predicted_aligned_error_v2.json
https://alphafold.ebi.ac.uk/files/AF-B0SWV2-F1-model_v2.pdb
codi no executat a Alphafold
3sovA02
O75581
https://alphafold.ebi.ac.uk/files/AF-O75581-F1-predicted_aligned_error_v2.json
https://alphafold.ebi.ac.uk/files/AF-O75581-F1-model_v2.pdb
6d9nB01
A0A077EDR1
https://alphafold.ebi.ac.uk/